In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from agent_load_testing.use_case_generator import generate_garage_use_case
from rlway_cpagent.osrd_adapter import regulation_problem_from_schedule, schedule_from_solution
from rlway_cpagent.ortools_agent.ortools_solver import OrtoolsRegulationSolver
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from agent_load_testing.load_testing import matrix_load_testing, linear_load_testing, load_testing

import nest_asyncio  
nest_asyncio.apply()

/home/simon/dev/python/cpagent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Large problem generation

In [6]:
problem = generate_garage_use_case(2, 2, 0.05)

## Solve

In [7]:
solver = OrtoolsRegulationSolver("ortools", 86400, True)
solution = solver.solve(problem)
regulated_schedule = schedule_from_solution(solution)

print(solution.status)
df = pd.DataFrame(solver.history, columns=['computation_time', 'objective', 'born_inf'])
df.to_csv("./large_problem.csv", index=False)

TypeError: schedule_from_solution() missing 1 required positional argument: 'solution'

## Viz Solution

In [5]:
df = pd.read_csv("./large_problem.csv")

In [ ]:
regulated_schedule.plot()

In [ ]:
times, delays, borns = zip(*solver.history)

In [ ]:
data = df.to_numpy()
times, delays, borns = data.T

In [ ]:

plt.plot(times/3600, delays/3600, label="ortools")
#plt.plot(times, borns, label="born_inf")
plt.axhline(y=borns[-1]/3600, color='r', linestyle='--', label="lower bound")
plt.xlabel("computation time (hours)")
plt.ylabel("sum of delays (hours)")
plt.legend()
plt.suptitle("(50 stations, 30 trains)", fontsize=10)
plt.title("Best found solution versus computationnal effort")
plt.show()

# Matrix Performance Testing

In [ ]:
performances = matrix_load_testing(
    [OrtoolsRegulationSolver("ortools",60),
     MinizincRegulationSolver("minizinc", "gecode", 60)],
    generate_garage_use_case,
    ranges= {
        "nb_stations": range(1, 18, 3),
        "nb_trains": range(1, 18, 3),
        "delay_proba": [0.1, 0.2]
    })

if not os.path.isfile('./performances.csv'):
   performances.to_csv("./performances.csv", header=True, index=False)
else:
   performances.to_csv("./performances.csv", mode='a', header=False, index=False)

### Ortools

In [ ]:
df = performances.pivot(index='nb_stations', columns='nb_trains', values='ortools')
#print(df)
df = df.sort_index(ascending=False)
print(df)

In [ ]:
ax = sns.heatmap(df)
plt.show()

### Minizinc

In [ ]:
df = performances.pivot(index='nb_stations', columns='nb_trains', values='minizinc')
#print(df)
df = df.sort_index(ascending=False)
print(df)

In [ ]:
ax = sns.heatmap(df)
plt.show()

### Linear Performance Testing

In [ ]:
NOMINAL_NB_TRAINS = 5
NOMINAL_NB_STATIONS = 3
NOMINAL_PROBA_DELAY = 0.1

In [ ]:
linear_performances = linear_load_testing(
    [OrtoolsRegulationSolver("ortools",60),
     MinizincRegulationSolver("minizinc", "gecode", 60)],
    generate_garage_use_case,
    ranges= {
        "nb_stations": {
            "range": list(range(1, 5, 2)) + list(range(5, 20, 4)),
            "nominal": NOMINAL_NB_STATIONS
        },
        "nb_trains": {
            "range": list(range(1, 5, 2)) + list(range(5, 20, 4)),
            "nominal": NOMINAL_NB_TRAINS
        },
        "delay_proba": {
            "range": [NOMINAL_PROBA_DELAY],
            "nominal": NOMINAL_PROBA_DELAY
        }
    })

if not os.path.isfile('./performances.csv'):
   linear_performances.to_csv("./performances.csv", header=True, index=False)
else:
   linear_performances.to_csv("./performances.csv", mode='a', header=False, index=False)

### Nb_Stations impact

In [ ]:
station_df = linear_performances[linear_performances['nb_trains'] == NOMINAL_NB_TRAINS]
station_df = station_df.drop_duplicates(subset=['nb_stations', 'nb_trains'])
station_df = station_df.sort_values(by="nb_stations", ascending=True)
plt.plot(station_df['nb_stations'], station_df['ortools'], label="ortools")
plt.plot(station_df['nb_stations'], station_df['minizinc'], label='minizinc')
plt.xlabel("nb_stations")
plt.ylabel("computation_time(s)")
plt.legend()

### Nb_Trains impact

In [ ]:
train_df = linear_performances[linear_performances['nb_stations'] == NOMINAL_NB_STATIONS]
train_df = train_df.drop_duplicates(subset=['nb_stations', 'nb_trains'])
train_df = train_df.sort_values(by="nb_trains", ascending=True)
plt.plot(train_df['nb_trains'], train_df['ortools'], label="ortools")
plt.plot(train_df['nb_trains'], train_df['minizinc'], label="minizinc")
plt.xlabel("nb_trains")
plt.ylabel("computation_time(s)")
plt.legend()

## Report from csv

In [ ]:
performances_df = pd.read_csv("./performances.csv")

In [ ]:
grouped_performances = performances_df.groupby(['nb_stations', 'nb_trains', 'delay_proba']).mean().reset_index()
heatmap_delay_0_1_df = grouped_performances[grouped_performances['delay_proba'] == 0.1].pivot(index='nb_stations', columns='nb_trains', values='ortools')
heatmap_delay_0_1_df.head()

In [ ]:
ax = sns.heatmap(heatmap_delay_0_1_df)
plt.show()

In [ ]:
heatmap_delay_0_2_df = grouped_performances[grouped_performances['delay_proba'] == 0.2].pivot(index='nb_stations', columns='nb_trains', values='ortools')
heatmap_delay_0_2_df.head()

In [ ]:
ax = sns.heatmap(heatmap_delay_0_2_df)
plt.show()

In [ ]:
performances_df.to_latex()

## Table

In [ ]:
df = load_testing(
    [OrtoolsRegulationSolver("ortools",3600)],
    generate_garage_use_case,
    [
        {'nb_stations':9, 'nb_trains':9, 'delay_proba':0.05}
    ],
    "./table2.csv"
)

In [ ]:
df = pd.read_csv("./table2.csv")
df.to_latex(index=False, float_format="%.2f")